## Challenge TelecomX-2-ML – Análisis Predictivo de Churn

### Objetivo

En esta segunda etapa del Challenge de Data Science de Alura, el enfoque se centra en el **modelado predictivo** para anticipar la evasión de clientes (Churn) en TelecomX.
Luego de completar el proceso de **ETL y Análisis Exploratorio (EDA)**, el objetivo ahora es construir, evaluar y optimizar modelos de Machine Learning que permitan:

* Predecir qué clientes tienen mayor probabilidad de abandonar el servicio.
* Identificar las variables más influyentes en la decisión de churn.
* Proporcionar recomendaciones estratégicas basadas en evidencia predictiva.


Flujo de implementación:

1. Feature Engineering
2. Preprocesamiento
3. Train/Test SplitPipeline
4. Cross-Validation (en Train)
5. Comparación de modelos
6. Optimización
7. Entrenamiento final
8. Evaluación en Test

---



## Implementación del Modelado Predictivo


### Importación de librerías y configuración del entorno

In [7]:
# ==============================
# CONFIGURACIÓN GLOBAL
# ==============================

# Ignorar warnings irrelevantes
import warnings
#warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


# ==============================
# BIBLIOTECAS PARA MANEJO DE DATOS
# ==============================

import requests
import pandas as pd
import numpy as np
import json
import os

# ==============================
# BIBLIOTECAS PARA PREPROCESAMIENTO - ML
# ==============================

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)

from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
# ==============================
# BIBLIOTECAS PARA VISUALIZACIÓN
# ==============================

import matplotlib.pyplot as plt
import seaborn as sns

# ==============================
# CONFIGURACIÓN DE VISUALIZACIONES
# ==============================

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Si trabajas en Jupyter, puedes activar esto:
# %matplotlib inline

# ==============================
# CONFIGURACIÓN DE PANDAS
# ==============================

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# ==============================
# PALETA DE COLORES PERSONALIZADA
# ==============================

color_no_churn = '#2C3E50'  # Azul oscuro elegante
color_churn = '#E74C3C'     # Rojo elegante

color_palette = [
  '#F8F9FA', # Fondo Limpio (Ghost White)
  '#E9ECEF', # Bordes/Secciones (Slate Gray)
  '#22577A', # Títulos/Estructura (Dark Imperial Blue)
  '#38A3A5', # Procesos activos (Cadet Blue)
  '#57CC99', # Éxito/Validación (Emerald)
  '#80ED99'  # Acentos ligeros (Light Green)
]

sns.set_palette(color_palette)

# ==============================
# CREAR CARPETA PARA IMÁGENES
# ==============================

os.makedirs('imgs', exist_ok=True)

print('✅ Bibliotecas importadas y entorno configurado correctamente')

✅ Bibliotecas importadas y entorno configurado correctamente


### Carga de datos

Previo análisis con dataset generado en la 1ra etapa y al no encontrar mayores inconsistencias, se utiliza el dataset proprocionado por Alura One "dataset_tratado.csv". 

In [10]:
url = "https://raw.githubusercontent.com/LenninTemoche/Challenge-TelecomX-2-Alura-One-DS/refs/heads/main/datos_tratados.csv"

df = pd.read_csv(url, sep=",")

df.sample(3)

,CustomerID,Churn,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,ChargesDaily,ChargesMonthly,ChargesTotal
5611,7901-TBKJX,No,Male,Yes,No,No,56,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,3.37,101.05,5594.00
3668,5176-LDKUH,No,Female,No,No,No,48,Yes,No,Fiber optic,No,Yes,No,No,No,No,One year,No,Electronic check,2.51,75.15,3772.65
4020,5668-MEISB,No,Female,No,Yes,Yes,72,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),3.54,106.10,7657.40


## 1. Limpieza de Datos

En esta etapa se garantiza la calidad del dataset antes del modelado.

Se realizaron las siguientes acciones:

- Estandarización de categorías inconsistentes.
- Eliminación de variables sin valor predictivo.
- Verificación final de valores nulos.
- Eliminación de registros duplicados (si existieran).

Este paso es fundamental para evitar ruido, sesgos y errores en el entrenamiento de los modelos.

In [11]:
# ============================================================
# 1️⃣ LIMPIEZA DE DATOS (con validaciones antes y después)
# ============================================================

import numpy as np
import pandas as pd

print("Iniciando limpieza de datos...")

# ------------------------------------------------------------
# 🔎 1.0 Diagnóstico inicial general
# ------------------------------------------------------------

print("\n📌 DIAGNÓSTICO INICIAL")

print(f"Filas iniciales: {df.shape[0]}")
print(f"Columnas iniciales: {df.shape[1]}")

# ---- Nulos iniciales ----
nulos_iniciales = df.isnull().sum().sum()
print(f"Nulos totales iniciales: {nulos_iniciales}")

# ---- Duplicados iniciales (con ID si existe) ----
duplicados_iniciales = df.duplicated().sum()
print(f"Duplicados iniciales (con ID): {duplicados_iniciales}")


# ------------------------------------------------------------
# 1.1 Estandarización de categorías
# ------------------------------------------------------------

print("\n🔄 Estandarizando categorías...")

cols_to_fix = [
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

for col in cols_to_fix:
    if col in df.columns:
        antes = df[col].value_counts(dropna=False)
        df[col] = df[col].replace({'No internet service': 'No'})
        despues = df[col].value_counts(dropna=False)

        print(f"\nColumna: {col}")
        print("Valores únicos antes:")
        print(antes)
        print("Valores únicos después:")
        print(despues)

print("\nCategorías estandarizadas correctamente.")


# ------------------------------------------------------------
# 1.2 Eliminación de variable identificadora
# ------------------------------------------------------------

print("\n🗑 Validando eliminación de ID...")

print(f"Columnas antes de eliminar ID: {df.shape[1]}")

if 'CustomerID' in df.columns:
    df = df.drop(columns=['CustomerID'])
    print("Se eliminó columna 'CustomerID'")
elif 'customerID' in df.columns:
    df = df.drop(columns=['customerID'])
    print("Se eliminó columna 'customerID'")
else:
    print("No se encontró columna identificadora.")

print(f"Columnas después de eliminar ID: {df.shape[1]}")

# ---- Validar duplicados después de quitar ID ----
duplicados_sin_id = df.duplicated().sum()
print(f"Duplicados después de eliminar ID: {duplicados_sin_id}")


# ------------------------------------------------------------
# 1.3 Eliminación de duplicados
# ------------------------------------------------------------

print("\n🧹 Eliminando duplicados (si existen)...")

filas_antes = df.shape[0]

if duplicados_sin_id > 0:
    df = df.drop_duplicates()
    filas_despues = df.shape[0]
    print(f"Se eliminaron {filas_antes - filas_despues} registros duplicados.")
else:
    print("No se encontraron registros duplicados.")

print(f"Filas actuales: {df.shape[0]}")


# ------------------------------------------------------------
# 1.4 Verificación final de nulos
# ------------------------------------------------------------

print("\n📌 VERIFICACIÓN FINAL")

nulos_finales = df.isnull().sum().sum()

reporte_final = pd.DataFrame({
    'Nulos': df.isnull().sum(),
    'Porcentaje (%)': (df.isnull().sum() / len(df)) * 100
}).sort_values(by='Nulos', ascending=False)

if nulos_finales > 0:
    print("Columnas con nulos:")
    print(reporte_final[reporte_final['Nulos'] > 0])
else:
    print("No hay valores nulos en el dataset.")

print(f"Nulos totales finales: {nulos_finales}")

print("\n✅ Limpieza de datos finalizada correctamente.")
print(f"Shape final del dataset: {df.shape}")

Iniciando limpieza de datos...

📌 DIAGNÓSTICO INICIAL
Filas iniciales: 7043
Columnas iniciales: 22
Nulos totales iniciales: 0
Duplicados iniciales (con ID): 0

🔄 Estandarizando categorías...

Columna: OnlineSecurity
Valores únicos antes:
OnlineSecurity
No     5024
Yes    2019
Name: count, dtype: int64
Valores únicos después:
OnlineSecurity
No     5024
Yes    2019
Name: count, dtype: int64

Columna: OnlineBackup
Valores únicos antes:
OnlineBackup
No     4614
Yes    2429
Name: count, dtype: int64
Valores únicos después:
OnlineBackup
No     4614
Yes    2429
Name: count, dtype: int64

Columna: DeviceProtection
Valores únicos antes:
DeviceProtection
No     4621
Yes    2422
Name: count, dtype: int64
Valores únicos después:
DeviceProtection
No     4621
Yes    2422
Name: count, dtype: int64

Columna: TechSupport
Valores únicos antes:
TechSupport
No     4999
Yes    2044
Name: count, dtype: int64
Valores únicos después:
TechSupport
No     4999
Yes    2044
Name: count, dtype: int64

Columna: Stre


### 1 Ingeniería de Características (Feature Engineering)

En esta fase se optimizan y crean variables para mejorar la capacidad predictiva del modelo.

* **Creación de `num_services`**: Variable que cuantifica la intensidad de uso del cliente. Un mayor número de servicios puede reflejar mayor fidelización o mayor complejidad de abandono.
* **Variables derivadas**: Posible creación de ratios como gasto promedio por servicio.
* **Eliminación de variables irrelevantes o redundantes**:

  * `CustomerID` (identificador sin valor predictivo).
  * Análisis de variables altamente correlacionadas que puedan generar multicolinealidad.
* **Análisis de importancia preliminar** para validar aporte de nuevas variables.

---

In [5]:
# Creación de la variable num_services
# 1. Definimos las columnas que representan servicios
service_cols = [
    'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
    'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies'
]

# 2. Función para contar los servicios activos de cada cliente
def count_services(row):
    count = 0
    # InternetService: Si es 'DSL' o 'Fiber optic' cuenta como 1 (si es 'No', es 0)
    if row['InternetService'] != 'No':
        count += 1
    
    # Para el resto, sumamos 1 si el valor es 'Yes'
    for col in service_cols:
        if col != 'InternetService' and row[col] == 'Yes':
            count += 1
    return count

# Aplicamos la función para crear la nueva columna
df['num_services'] = df.apply(count_services, axis=1)

print(f"✅ Nueva columna creada: 'num_services' (Rango: {df['num_services'].min()} - {df['num_services'].max()} servicios)")



✅ Nueva columna creada: 'num_services' (Rango: 1 - 9 servicios)


#### Creación de la variable `num_services`

Se construyó una nueva variable que cuantifica la cantidad de servicios activos contratados por cada cliente.

La lógica aplicada fue:

- Si el cliente posee Internet (DSL o Fiber optic), se considera 1 servicio.
- Se suma 1 por cada servicio adicional activo (`Yes`).
- Servicios sin contratación o con valor "No internet service" no se contabilizan.

Esta variable busca capturar la intensidad de uso del cliente, ya que un mayor número de servicios puede estar asociado con mayor fidelización o mayor complejidad de abandono.

In [6]:
df = df.drop(columns=['CustomerID'])

KeyError: "['CustomerID'] not found in axis"

### Eliminación de variable identificadora

Se eliminó la columna `CustomerID` debido a que es un identificador único sin valor predictivo.

Mantenerla podría introducir ruido o sobreajuste en modelos basados en árboles.



### 2 Preprocesamiento de Datos (Data Preprocessing)

Preparación técnica de los datos para su uso en modelos de Machine Learning.

* **Codificación de Variables Categóricas**:

  * One-Hot Encoding para variables nominales (ej. `PaymentMethod`).
  * Label Encoding para la variable objetivo (`Churn`: 0 = No, 1 = Sí).

* **Escalado de Variables Numéricas**:

  * Estandarización (StandardScaler) o Normalización (MinMaxScaler).
  * Aplicado a variables como:

    * `Tenure`
    * `ChargesMonthly`
    * `num_services`



In [ ]:


# 2. Codificación de la variable objetivo (Churn: Yes/No -> 1/0)
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

# 3. Identificación de variables para el ColumnTransformer
numeric_cols = ['Tenure', 'ChargesMonthly', 'num_services']
categorical_cols = [col for col in df.columns if col not in numeric_cols + ['Churn']]

# 4. Simplificación de valores en variables categóricas (No internet service -> No)

cols_to_fix = [
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

for col in cols_to_fix:
    df[col] = df[col].replace({'No internet service': 'No'})

# El siguiente paso sería aplicar el Pipeline con OneHotEncoder y MinMaxScaler
print("\n✅ Columnas redundantes eliminadas y dataset listo para encoding/scaling.")

### Estandarización de categorías

Se reemplazaron los valores "No internet service" por "No" en variables relacionadas con servicios adicionales.

Esto evita crear categorías artificiales al aplicar One-Hot Encoding y simplifica la interpretación del modelo.

In [ ]:
df_numeric = df.select_dtypes(include=["int64", "float64"])

df_numeric["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})

corr_matrix = df_numeric.corr()

# Visualizar heatmap
plt.figure(figsize=(10,6))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matriz de Correlación - Variábles Numericas")
plt.show()

### Matriz de correlación

Se analizó la correlación entre variables numéricas y la variable objetivo.

Este análisis permite identificar:

- Variables con mayor asociación lineal con churn.
- Posible multicolinealidad.
- Variables con bajo poder explicativo.